# Geocomputation 1, Lab 2
### Ian Grant

## Importing the data

In [127]:
import requests, json

In [128]:
# May not run. Check the online source.
url = "https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_5m.json"

with requests.get(url) as response:
    data = response.json()
    # Two different ways of composing text messages. The format function is preferred.
    print('Number of Counties in US: ' + str(len(data['features'])))
    print('First entry is {}'.format(data['features'][0]['properties']['NAME']))

Number of Counties in US: 3221
First entry is Valdez-Cordova


## Analysis of data file

### Top-level structure
First, I analyzed the structure of the data file in preparation for further analysis.

The following analysis shows that the top-level structure is a dictionary with two keys, 'type' and 'features':
    - The value for 'type' is the string 'FeatureCollection'; this likely indicates the overall type of the data structure stored in 'data'.
    - The value for 'features' is a list containing the county data, discussed further below.

In [129]:
print(f"'data' is a data structure of type: {type(data)}")
print(f"The keys of the 'data' dictionary are: {data.keys()}\n")

print(f"The value for the 'type' key in the 'data' dictionary is of type: {type(data['type'])}")
print(f"The value corresponding to the 'type' key is: {data['type']}\n")

print(f"The value for the 'features' key in 'data' dict is of type: {type(data['features'])}\n")



'data' is a data structure of type: <class 'dict'>
The keys of the 'data' dictionary are: dict_keys(['type', 'features'])

The value for the 'type' key in the 'data' dictionary is of type: <class 'str'>
The value corresponding to the 'type' key is: FeatureCollection

The value for the 'features' key in 'data' dict is of type: <class 'list'>



### Features list
Next, I examined the structure of the 'features' list.

First, I look at the beginning of the list to observe its structure.
It looks like the first element is a dictionary that corresponds to one county, since we can see 'NAME': 'Valdez-Cordov' and 'STATE: '02''
I confirmed this by looking at the types of the first two elements in the list; both are dictionaries. 

In [130]:
#Look at just the first 2000 characters of 'features' to evaluate the structure (otherwise it is quite long)
features_excerpt = str(data['features'])[:2000] + "\n"
print(features_excerpt)

print(f"Types of first and second elements in 'features' (respectively): {type(data['features'][0]), type(data['features'][1])}\n")


[{'type': 'Feature', 'properties': {'GEO_ID': '0500000US02261', 'STATE': '02', 'COUNTY': '261', 'NAME': 'Valdez-Cordova', 'LSAD': 'CA', 'CENSUSAREA': 34239.88}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-147.483828, 60.618636], [-147.500009, 60.653852], [-147.483828, 60.683358], [-147.487635, 60.728092], [-147.395312, 60.74332], [-147.383891, 60.741417], [-147.362, 60.714767], [-147.3087, 60.665274], [-147.348675, 60.627202], [-147.454323, 60.619588], [-147.483828, 60.618636]]], [[[-147.341061, 60.305499], [-147.340109, 60.275042], [-147.483828, 60.224598], [-147.499057, 60.235067], [-147.505719, 60.253151], [-147.496201, 60.265524], [-147.421962, 60.279801], [-147.341061, 60.305499]]], [[[-147.217704, 60.293504], [-147.19494, 60.304563], [-147.183277, 60.32068], [-147.185243, 60.323083], [-147.195608, 60.326224], [-147.211625, 60.324936], [-147.215312, 60.327109], [-147.218799, 60.334726], [-147.214679, 60.343793], [-147.211582, 60.34626], [-147.147514, 60.37247], [-147.

### Properties dictionaries

Finally, I explored the structure of the dictionaries corresponding to counties.
The keys at the county level are 'type', 'properties', and 'geometry'.
For this assignment, we probably do not need the coordinates stored in 'geometry'. I confirm this by observing the value for the "properties" key; it has everything needed for this assignment.

In [131]:
first_county = data['features'][0]
print(first_county.keys())
print(type(first_county['properties']))
print(first_county['properties'])

dict_keys(['type', 'properties', 'geometry'])
<class 'dict'>
{'GEO_ID': '0500000US02261', 'STATE': '02', 'COUNTY': '261', 'NAME': 'Valdez-Cordova', 'LSAD': 'CA', 'CENSUSAREA': 34239.88}


## Data processing and function definition

### Extracting the 'properties' dictionaries

Based on the results of the analysis above, we only need to deal with the 'properties' dictionaries within 'data'. This is a relatively small part of the data file, since most of the data is geometric coordinates. Therefore, the first step is to make a list of just the 'properties' dictionaries.

In [132]:
counties_props = []

for county in data['features']:
    counties_props.append(county['properties'])

#We no longer need to store the whole input file in 'data', but I have left this commented for convenience
#del(data)

### Aggregating the counties

First, I define a function called aggregateCounties() that will return a dictionary that aggregates the counties according to the values of a specified property.

The keys of the dictionary are all the unique values of the specified property.

The value for each key is a list containing the properties dictionary of each county that matches the key.

For example, if we aggregate by state, the function will return a dict where the keys are the state names and the values are lists of counties that fall within each state (see the analysis by state below).

In [133]:
def aggregateCounties(county_props_list, key_prop):
    agg_dict = {}

    #create a set of all the unique values of key_prop
    unique_props = set([county[key_prop] for county in county_props_list])
    
    #initialize dictionary with unique values as keys and empty lists as values
    for unique_prop in unique_props:
        agg_dict[unique_prop] = []

    #update the values for each key in the dict
    for county in county_props_list:
        #append the properties dict of the current county to the value of the appropriate agg_dict key
        agg_dict[county[key_prop]].append(county)
    
    return agg_dict

### Calculating the most frequent values

Next, we'll make a function that uses an aggegated dictionary to return a sorted list of the n keys with the most counties.

Taking our state example above, we could use this function with the dictionary aggregated by state to return the five states with the most counties.

In [134]:
def mostCommonProps(agg_dict, n):

    #cap n at the length of the dictionary
    if n > len(agg_dict):
        n = len(agg_dict)

    common_props = []
    
    #Make a 2D array holding the n most common values and the number of counties for each value
    #This array will update as we search the dict 
    for i in range(n):
        common_props.append([None, 0])

    for key in agg_dict:
        #calculate the number of counties in the current key of the aggregated dictionary
        num_counties = len(agg_dict[key])

        common_props_counts = [common_prop[1] for common_prop in common_props]

        if num_counties > min(common_props_counts):
            index = common_props_counts.index(min(common_props_counts))
            common_props[index] = [key, num_counties]

    common_props = [key[0] for key in sorted(common_props, key=lambda row : row[1], reverse=True)]
    return common_props

### State codes dictionary

Before we continue, let's define a dictionary of all the state codes so that we can display the states by name.

In [135]:
state_codes_def = """\
01,Alabama
02,Alaska
04,Arizona
05,Arkansas
06,California
08,Colorado
09,Connecticut
10,Delaware
11,District of Columbia
12,Florida
13,Georgia
15,Hawaii
16,Idaho
17,Illinois
18,Indiana
19,Iowa
20,Kansas
21,Kentucky
22,Louisiana
23,Maine
24,Maryland
25,Massachusetts
26,Michigan
27,Minnesota
28,Mississippi
29,Missouri
30,Montana
31,Nebraska
32,Nevada
33,New Hampshire
34,New Jersey
35,New Mexico
36,New York
37,North Carolina
38,North Dakota
39,Ohio
40,Oklahoma
41,Oregon
42,Pennsylvania
44,Rhode Island
45,South Carolina
46,South Dakota
47,Tennessee
48,Texas
49,Utah
50,Vermont
51,Virginia
53,Washington
54,West Virginia
55,Wisconsin
56,Wyoming
72,Puerto Rico"""

#Parse defintion string into 2D list of format [[code1, state1], [code2, state2], etc.]
state_codes_def = state_codes_def.split('\n')
for i in range(len(state_codes_def)):
    state_codes_def[i] = state_codes_def[i].split(',')

#Create state codes dictionary
state_codes = {}
for pair in state_codes_def:
    state_codes[pair[0]] = pair[1]

print(f"State code '51' corresponds to {state_codes['51']}.")

State code '51' corresponds to Virginia.


## County names analysis

We can use these functions to aggregate counties by county name and then look at the three most frequently occuring counties. 

We can see that "Washington" is first, "Jefferson" is second, and "Franklin" is third, and we can verify this by checking the number of counties that appear for each key. 

In [136]:
county_dict = aggregateCounties(counties_props, 'NAME')
common_county_names = mostCommonProps(county_dict, 3)

for county_name in common_county_names:
    print(f"There are {len(county_dict[county_name])} {county_name} Counties.")

There are 31 Washington Counties.
There are 26 Franklin Counties.
There are 26 Jefferson Counties.


We can use the resulting list of county names to get more information about those counties.

In [137]:
#Iterate through list of most common names
for county_name in common_county_names:
    #Print states where each county name appears
    print(f"There are {len(county_dict[county_name])} {county_name} Counties, one each in the following states:\n {[state_codes[county['STATE']] for county in county_dict[county_name]]} \n")

There are 31 Washington Counties, one each in the following states:
 ['Colorado', 'Florida', 'Arkansas', 'Illinois', 'Iowa', 'Maine', 'Maryland', 'Missouri', 'Tennessee', 'Oklahoma', 'Oregon', 'Wisconsin', 'Utah', 'Vermont', 'Kansas', 'Indiana', 'Kentucky', 'Minnesota', 'Georgia', 'Alabama', 'Louisiana', 'Idaho', 'New York', 'North Carolina', 'Nebraska', 'Mississippi', 'Pennsylvania', 'Ohio', 'Rhode Island', 'Virginia', 'Texas'] 

There are 26 Franklin Counties, one each in the following states:
 ['Georgia', 'Alabama', 'Indiana', 'Idaho', 'Kentucky', 'Iowa', 'Mississippi', 'New York', 'Texas', 'Arkansas', 'Florida', 'Louisiana', 'Maine', 'Massachusetts', 'Illinois', 'Kansas', 'Missouri', 'Tennessee', 'Nebraska', 'North Carolina', 'Pennsylvania', 'Ohio', 'Virginia', 'Virginia', 'Washington', 'Vermont'] 

There are 26 Jefferson Counties, one each in the following states:
 ['Illinois', 'Georgia', 'Iowa', 'Oregon', 'Colorado', 'Arkansas', 'Florida', 'Kansas', 'Kentucky', 'Alabama', 'Louisi

Interestingly, there are two Franklin Counties for state code 51 (Virginia). I wanted to be sure this was not an error in my analysis, so I checked the corresponding entries in the counties_props list. 

We can see that there are indeed two distinct county entries that differ in the 'LSAD' property: one is marked as a 'County' and the other is marked as a 'city'. 

In [138]:
ans = []

for county in counties_props:
    #print(county['NAME'])
    if county['NAME'] == 'Franklin' and county['STATE'] == '51':
        ans.append(county)

for county in ans:
    print(county)


{'GEO_ID': '0500000US51067', 'STATE': '51', 'COUNTY': '067', 'NAME': 'Franklin', 'LSAD': 'County', 'CENSUSAREA': 690.426}
{'GEO_ID': '0500000US51620', 'STATE': '51', 'COUNTY': '620', 'NAME': 'Franklin', 'LSAD': 'city', 'CENSUSAREA': 8.206}


LSAD stands for 'Legal/Statistical Area Description'. It is a US Census designation that describes types of entities.

The fact that there are two Franklins in Virginia, one with LSAD "County" and the other "city", raises a question about the data: why are some entries listed as LSAD type "County" but others not? How many LSAD types are included in this dataset?

We can use the same methods as above to anwer these questions.

In [139]:
LSAD_dict = aggregateCounties(counties_props, 'LSAD')
for LSAD in LSAD_dict:
    print(f"There are {len(LSAD_dict[LSAD])} counties of type {LSAD}.")

There are 2 counties of type .
There are 41 counties of type city.
There are 64 counties of type Parish.
There are 78 counties of type Muno.
There are 2 counties of type Muny.
There are 12 counties of type Borough.
There are 11 counties of type CA.
There are 3007 counties of type County.
There are 4 counties of type Cty&Bor.


We can see that there are 3,007 entries of type "County," as we would expect, but more than a hundred entries with other LSAD values. We would have to know more about the dataset to understand why, but it's interesting to note that not all of the putative counties are listed as such.

## State-by-state analysis

Next, we will examine state-level statistics. We can apply the same functions as above but aggregate on state code rather than county name. 

We can also see the top 5 states with the most counties using the same function as above.

In [140]:
#Create dictionary aggregated by state code
state_dict = aggregateCounties(counties_props, 'STATE')

print(f"The five states with the most counties are: {[state_codes[state] for state in mostCommonProps(state_dict, 5)]})")

The five states with the most counties are: ['Texas', 'Georgia', 'Virginia', 'Kentucky', 'Missouri'])


Next, we can use the state dictionary to calculate state statistics, writing the results to an output file.

In [141]:
#open the output file for writing (which clears the existing data in the file)
output_file = open("counties_analysis_by_state.txt", "w")

for state_code in state_dict:

    #state is the list of all counties in the state corresponding to state_code
    state = state_dict[state_code]

    #calculate how many counties are in this state
    num_counties = len(state)

    #get list of just the county areas in this state
    area_list = [county['CENSUSAREA'] for county in state]

    #calculate the mean of the county areas
    mean_area = sum(area_list) / len(area_list)

    #identify largest and smallest counties
    smallest_county = state[area_list.index(min(area_list))]
    biggest_county = state[area_list.index(max(area_list))]
    
    output_file.write(f"The mean area of the {num_counties} counties in {state_codes[state_code]} is {round(mean_area, 2)}.\n")
    output_file.write(f"The smallest is {smallest_county['NAME']} County, with an area of {smallest_county['CENSUSAREA']}.\n")
    output_file.write(f"The largest is {biggest_county['NAME']} County, with an area of {biggest_county['CENSUSAREA']}.\n\n")
    

#close the output file
output_file.close()

#open the output file again for reading
output_file = open("counties_analysis_by_state.txt", "r")

#print the first 1000 characters of the output file
print(output_file.read(1000))

output_file.close()



The mean area of the 44 counties in Idaho is 1878.25.
The smallest is Payette County, with an area of 406.867.
The largest is Idaho County, with an area of 8477.352.

The mean area of the 64 counties in Louisiana is 675.06.
The smallest is Orleans County, with an area of 169.423.
The largest is Vernon County, with an area of 1327.91.

The mean area of the 66 counties in South Dakota is 1148.65.
The smallest is Clay County, with an area of 412.185.
The largest is Meade County, with an area of 3470.984.

The mean area of the 62 counties in New York is 760.1.
The smallest is New York County, with an area of 22.829.
The largest is St. Lawrence County, with an area of 2680.377.

The mean area of the 56 counties in Montana is 2599.03.
The smallest is Silver Bow County, with an area of 718.477.
The largest is Beaverhead County, with an area of 5541.624.

The mean area of the 39 counties in Washington is 1703.99.
The smallest is San Juan County, with an area of 173.915.
The largest is Okanogan